# Regression Practice

-- [SICSS Zürich 2021](https://github.com/computational-social-science-zurich/sicss-zurich) -- 

## Preambule

### <span style='color:green'> Questions to answer are in green</span>

## Objective of the Notebook
This notebook walks you through the estimation of the **gender pay gap** using a combination of linear regressions and regularization models. 

Policy makers have long been concerned with the gender earnings gap. We will examine the gender earnings gap using data from the 2018 Current Population Survey (CPS) in the US. In particular, we will use the version of the CPS provided by the NBER.

To be more concrete, consider the following regression model:
$$y=\theta d+f(x) +\epsilon,$$ 
Where
- $d$ is the regressor of interest: a `female` dummy
- $y_i$ is a the outcome of interest: labour earnings or wage
- $x_i$ is a set of controls

We are not interested in $x$ per se (but we need to include it to avoid omitted variable bias)- and $f(x)$ is some unknown function. Instead we are interested in the estimation of $\theta$. We will estimate how x affect both $y_i$ and $d_i$ using a regularized model before estimating the relating between $y_i$ and $d_i$ with a linear regression. In other words, we will use a machine learning model to estimate $f$ in an automated and data-driven way.  

This model is called a **partually linear model**: linear in $d$ but not in $x$. 


### Estimation procedure for $\theta$:
1. Predict $y$ and $d$ from $x$ using a machine learning method relying on cross-validation
2. Partial out $x$: let $\tilde y_i = y_i - \hat y_i$ and $\tilde d_i=d_i - \hat d_i$
3. Regress $\tilde y_i$ on $\tilde d_i$ to estimate $\hat \theta$

The notebook will walk you through the previous steps. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
#plt.style.use('tableau-colorblind10')
#plt.style.use('Solarize_Light2')
plt.style.use('bmh')
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

## 0. Download CPS data

In [ ]:
cpsall = pd.read_stata("https://www.nber.org/morg/annual/morg18.dta")

In [ ]:
# take subset of data just to reduce computation time
cps = cpsall.sample(30000, replace=False, random_state=0)
display(cps.head())
cps.describe()

The variable `earnwke` records weekly earnings. Two variables detail the hours of work. `uhours` is usual hours worked per week, and `hourslw` is hours worked last week. We will try using each measure of hours to construct the wage. 

Let's define some variables common to everyone:

In [ ]:
# d variable:
cps["female"] = (cps['sex']==2)

# y variable:
cps["log_earn"] = np.log(cps['earnwke'])
cps["log_earn"][np.isinf(cps['log_earn'])] = np.nan

# hours (usual):
cps["log_uhours"] = np.log(cps['uhourse'])
cps["log_uhours"][np.isinf(cps['log_uhours'])] = np.nan

# hours (last week):
cps["log_hourslw"] = np.log(cps.hourslw)
cps["log_hourslw"][np.isinf(cps.log_hourslw)] = np.nan

# wage = earnigns / hours
cps["log_wageu"] = cps['log_earn'] - cps['log_uhours']

# <span style='color:green'>1. Raw difference in earnings by gender </span>
###  <span style='color:green'> 1.a. Estimate the unconditional gender `earnings` and `wage gaps` (that is just the log earnings of male - log earnings of female). </span>

### <span style='color:green'>1.b. Comment on the result: How would you explain the difference? </span>

# 2. Equal Pay for Equal Work?

A common slogan is equal pay for equal work. One way to interpret this is that for employees with similar worker and job characteristics, no gender wage gap should exist. 

Let’s examine whether there is a gender wage gap conditional on all worker and job characteristics. 
To ensure that we control for those characteristics as flexibly as possible, we will use the **partially linear model** described above.

Install useful packages

In [ ]:
!pip install patsy
!pip install statsmodels

###  Step 0. Create X, y and d
We keep the following varibles in $X$: hours, age, race, location, education, union membership, industry, and occupation

In [ ]:
from patsy import dmatrices
# Prepare data
fmla  = "log_earn + female ~ log_uhours + log_hourslw + age + I(age**2) + C(race) + C(cbsafips) + C(smsastat) + C(grade92) + C(unionmme) + C(unioncov) + C(ind02) + C(occ2012)"
yd, X = dmatrices(fmla,cps)
female = yd[:,1]
logearn = yd[:,2]

###  <span style='color:green'> Step 1. Predict $y$ and $d$ from $x$ using ant machine learning method relying on cross-validation

Estimate two `LASSO` for predicting $y$ (log earnings) and $d$ (female) from $x$ using a machine learning method relying on cross-validation. </span>
####  <span style='color:green'> Step 1.a. Estimate each model with cross-validation of a range of lambda </span>

Do not forget to scale the $X$ using `StandardScaler` for example

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import  LassoCV
import statsmodels as sm

####  <span style='color:green'> Step 1.b. Plot the test MSE as a function of $\alpha$ in order to choose the best $\alpha$ for both outcomes

####  <span style='color:green'> Step 1.c. <span style='color:green'>  Estimation of `yhat` and `dhat` using the best alpha for both outcomes estimted by cross validation previously </span>
    
You can predict $\hat y$ and $\hat d$ using `cross_val_predict` for each lasso model and a 5-fold cross-validation. 

##  Step 2:    Partial out $x$: 
    
Let $\tilde y_i = y_i - \hat y_i$ and $\tilde d_i=d_i - \hat d_i$


In [ ]:
ey = np.array(logearn - yhat)
ed = np.array(female - dhat)

## <span style='color:green'>  Step 3: Regress $\tilde y_i$ on $\tilde d_i$ to estimate $\hat \theta$
    
You can use 
`linear_model.OLS(y_variable,x_varible).fit(cov_type='HC0')` instead of `LinearRegression()` from `scikit-learn` to compute better standard errors (heteroscedastic robust: HC0)
   

# <span style='color:green'>3. Visualization of the results

## <span style='color:green'>  3.a. Create a dataframe with log earnings, predicted log earnings ($\hat y_i$, by the lasso), female and predicted $d$ ($\hat d_i$ by the lasso) 

## <span style='color:green'>  3.b. Using the dataframe, plot the $y$ vs. $y_hat$ ie. observed and predicted log(earnings) for different colors for male and female. 
You can for example use `pairplot` from `seaborn`(the `hue` options allows you to attribute the color depending on a variable).

## <span style='color:green'>  3.c. Plot the distribution of errors ($y-\hat y$) with $\hat y$ on the x-axis with colors depending on gender. 
You can use a simple `scatterplot` from seaborn.

# <span style='color:green'> 4. Comment on the results
    
<span style='color:green'> What is the estimated gender earnings gap conditional on $X$? How does it compare to the unconditional gender gap? 